In [1]:
import openmm as mm
from openmm import unit
from openmm import app

In [61]:
pdb = app.PDBFile('barnase_barstar.pdb')

In [3]:
type(pdb) 

openmm.app.pdbfile.PDBFile

In [4]:
dir(pdb)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_atomNameReplacements',
 '_loadNameReplacementTables',
 '_numpyPositions',
 '_parseResidueAtoms',
 '_positions',
 '_residueNameReplacements',
 '_standardResidues',
 'getNumFrames',
 'getPositions',
 'getTopology',
 'positions',
 'topology',
 'writeFile',
 'writeFooter',
 'writeHeader',
 'writeModel']

In [15]:
positions = pdb.getPositions(asNumpy=True)

In [20]:
positions._value.shape

(3159, 3)

In [48]:
list_chains = []
for chain in pdb.topology.chains():
    list_chains.append(chain)

In [50]:
chain_0 = list_chains[0]

In [55]:
list_atoms_in_chain_0 = []
for atom in chain_0.atoms():
    list_atoms_in_chain_0.append(atom)

In [56]:
len(list_atoms_in_chain_0)

1727

In [57]:
chain_1 = list_chains[1]
list_atoms_in_chain_1 = []
for atom in chain_1.atoms():
    list_atoms_in_chain_1.append(atom)
    
len(list_atoms_in_chain_1)

1432

In [62]:
original_positions = pdb.getPositions(asNumpy=True)

In [63]:
original_positions[0:1726]

Quantity(value=array([[-0.4722,  1.6032, -0.5051],
       [-0.4682,  1.66  , -0.6107],
       [-0.3589,  1.5705, -0.4824],
       ...,
       [ 0.4141,  3.4026,  0.6075],
       [ 0.3763,  3.3302,  0.5218],
       [ 0.5124,  3.4458,  0.5541]]), unit=nanometer)

In [64]:
original_positions[1727:]

Quantity(value=array([[-0.7268,  0.0096,  2.9545],
       [-0.7154, -0.074 ,  2.8696],
       [-0.8343,  0.0571,  2.9326],
       ...,
       [-0.6949,  0.7494,  4.3906],
       [-0.7008,  0.9331,  4.6818],
       [-0.7069,  0.8351,  4.7805]]), unit=nanometer)

In [71]:
centro_0 = np.mean(original_positions[0:1726], 0)
centro_1 = np.mean(original_positions[1727:], 0)

In [72]:
centro_1-centro_0

Quantity(value=array([ 0.09376228, -1.11701723,  2.11753116]), unit=nanometer)

In [22]:
forcefield = app.ForceField('amber14-all.xml')

In [23]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

In [24]:
step_size   = 0.002*unit.picoseconds
temperature = 0.0*unit.kelvin
friction    = 0.0/unit.picosecond # Damping para la dinámica de Langevin

integrator = mm.LangevinIntegrator(temperature, friction, step_size)

In [25]:
platform_name = 'CPU'
platform    = mm.Platform.getPlatformByName(platform_name)

In [26]:
context = mm.Context(system, integrator, platform)

In [27]:
type(context)

openmm.openmm.Context

In [28]:
context.setPositions(pdb.positions)

In [29]:
state = context.getState(getEnergy=True)

In [30]:
state.getPotentialEnergy()


Quantity(value=15294.695710703127, unit=kilojoule/mole)

In [32]:
coordenadas = pdb.getPositions(asNumpy=True)

In [39]:
coordenadas._value[0,0]=-1000.0
coordenadas._value[0,1]=0.0
coordenadas._value[0,2]=0.0

In [40]:
coordenadas

Quantity(value=array([[-1.0000e+03,  0.0000e+00,  0.0000e+00],
       [-4.6820e-01,  1.6600e+00, -6.1070e-01],
       [-3.5890e-01,  1.5705e+00, -4.8240e-01],
       ...,
       [-6.9490e-01,  7.4940e-01,  4.3906e+00],
       [-7.0080e-01,  9.3310e-01,  4.6818e+00],
       [-7.0690e-01,  8.3510e-01,  4.7805e+00]]), unit=nanometer)

In [41]:
context.setPositions(coordenadas)

In [42]:
state = context.getState(getEnergy=True)
state.getPotentialEnergy()

Quantity(value=153340498826.4681, unit=kilojoule/mole)